In [72]:
import pandas as pd

In [73]:
df = pd.read_csv("fixation_data.tsv", sep="\t")
df.tail()

,Recording timestamp,Computer timestamp,Sensor,Project name,Participant name,Recording name,Recording duration,Timeline name,Recording Fixation filter name,Recording software version,...,Fixation point X,Fixation point Y,Fixation point X (MCSnorm),Fixation point Y (MCSnorm),Viewport position X,Viewport position Y,Viewport width,Viewport height,Mouse position X,Mouse position Y
25603,101270352,7647883560,Eye Tracker,Gaze_model1,Sergiy,Recording2,101451,Timeline1,Tobii I-VT (Fixation),1.145.28180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25604,101278727,7647891935,Eye Tracker,Gaze_model1,Sergiy,Recording2,101451,Timeline1,Tobii I-VT (Fixation),1.145.28180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25605,101287015,7647900223,Eye Tracker,Gaze_model1,Sergiy,Recording2,101451,Timeline1,Tobii I-VT (Fixation),1.145.28180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25606,101306156,7647919364,NaN,Gaze_model1,Sergiy,Recording2,101451,Timeline1,Tobii I-VT (Fixation),1.145.28180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25607,101450654,7648063862,NaN,Gaze_model1,Sergiy,Recording2,101451,Timeline1,Tobii I-VT (Fixation),1.145.28180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean up data (Columns ect.) Skriver df.columns og får opp alle kolonnene. 

In [74]:
df.columns

Index(['Recording timestamp', 'Computer timestamp', 'Sensor', 'Project name',
       'Participant name', 'Recording name', 'Recording duration',
       'Timeline name', 'Recording Fixation filter name',
       'Recording software version', 'Recording resolution height',
       'Recording resolution width', 'Recording monitor latency',
       'Average calibration accuracy (mm)',
       'Average calibration precision SD (mm)',
       'Average calibration precision RMS (mm)',
       'Average calibration accuracy (degrees)',
       'Average calibration precision SD (degrees)',
       'Average calibration precision RMS (degrees)',
       'Average calibration accuracy (pixels)',
       'Average calibration precision SD (pixels)',
       'Average calibration precision RMS (pixels)',
       'Average validation accuracy (mm)',
       'Average validation precision SD (mm)',
       'Average validation precision RMS (mm)',
       'Average validation accuracy (degrees)',
       'Average validation 

In [56]:
cols_to_keep= ['Recording timestamp', "Participant name", "Recording name", "Eye movement type", "Gaze event duration", "Fixation point X (MCSnorm)", "Fixation point Y (MCSnorm)", "Presented Media name", "Eye movement type index"]

In [75]:
df = df[cols_to_keep]

In [76]:
df.tail()

,Recording timestamp,Participant name,Recording name,Eye movement type,Gaze event duration,Fixation point X (MCSnorm),Fixation point Y (MCSnorm),Presented Media name,Eye movement type index
25603,101270352,Sergiy,Recording2,Saccade,25.0,NaN,NaN,test1.png,1251.0
25604,101278727,Sergiy,Recording2,EyesNotFound,8.0,NaN,NaN,test1.png,1360.0
25605,101287015,Sergiy,Recording2,Unclassified,8.0,NaN,NaN,test1.png,1618.0
25606,101306156,Sergiy,Recording2,NaN,NaN,NaN,NaN,NaN,NaN
25607,101450654,Sergiy,Recording2,NaN,NaN,NaN,NaN,NaN,NaN


Filtrerer, ved å legge inn boolsk condition i df[], og får true-radene. Filtrer bort kalibrering.

In [77]:
df =  df[df['Presented Media name']=="test1.png"]
df = df[df["Eye movement type"]=="Fixation"]

In [ ]:
df_eskil = df["Participant name" == "Sergiy"]

In [78]:
df.tail()

,Recording timestamp,Participant name,Recording name,Eye movement type,Gaze event duration,Fixation point X (MCSnorm),Fixation point Y (MCSnorm),Presented Media name,Eye movement type index
25558,100895419,Sergiy,Recording2,Fixation,108.0,"0,4721","0,3226",test1.png,188.0
25559,100903725,Sergiy,Recording2,Fixation,108.0,"0,4721","0,3226",test1.png,188.0
25560,100912075,Sergiy,Recording2,Fixation,108.0,"0,4721","0,3226",test1.png,188.0
25561,100920425,Sergiy,Recording2,Fixation,108.0,"0,4721","0,3226",test1.png,188.0
25562,100928718,Sergiy,Recording2,Fixation,108.0,"0,4721","0,3226",test1.png,188.0


Alle fixations rett etter hverandre med lik verdi, og likt fokuseringspunkt, må slås sammen

In [79]:
df.drop_duplicates(subset="Eye movement type index", inplace = True) 

In [87]:
df.tail()

,Recording timestamp,Participant name,Recording name,Eye movement type,Gaze event duration,Fixation point X (MCSnorm),Fixation point Y (MCSnorm),Presented Media name,Eye movement type index
21388,66148117,Sergiy,Recording2,Fixation,67.0,"0,7198","0,2021",test1.png,95.0
21424,66448096,Sergiy,Recording2,Fixation,67.0,"0,7039","0,2285",test1.png,96.0
21447,66639797,Sergiy,Recording2,Fixation,108.0,"0,6568","0,3088",test1.png,97.0
21480,66914719,Sergiy,Recording2,Fixation,108.0,"0,6868","0,3172",test1.png,98.0
21499,67073041,Sergiy,Recording2,Fixation,158.0,"0,6823","0,2958",test1.png,99.0


Videre: Fant ut feil. Må få splittet Sergiy og Eskil radene i starten, og lage eskil dataframe og Segiy dataframe, og deretter jobbe videre med dataene